# gempy test 


## Convert stratigraphy GML to time-sorted groups and units

<font face color='red'><b>RUN the "map2model_after_topology" notebook first!!<b></font>

Takes GML file produced by topology code, geology polygons, structure points and dtm to create 3D model.<br><br>
Limitations: no faults yet, no plutons yet, no dykes, no sills. Faults and plutons will come soon, sills require us to assign a unique surface to each instance of a sill (sill between units A and B needs to be different from sill of same age and strat codes as one found between E and F). Dykes via cokriging are really hard without just cookie cutting them in (but that is not our problem!). We are not checking for onlap relationships, which can perhaps been seen by having lots of units from one series adjacent to the youngest surface of the older series. Could also think about interpreting these as faults to introduce conceptual uncertainty.

<font face color='red'>The way this loads data to gempy is not right, so all the units end up in one series... but it gives the basic idea </font>


In [ ]:
import matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import numpy as np
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import os
import rasterio
from rasterio.transform import from_origin
from rasterio import features
%matplotlib inline
import sys, os
print(os.getcwd())

os.chdir('../map2loop')
import m2l_utils
print(os.getcwd())
os.chdir('../')


## Now we define the basic model and load the orientation and lithological information
mj: We inset the model dimensions as once reprojects the edge pixels of the topography are undefined. Probably doesn't need to be that extreme (unit is metres). Could use parameter <b>maxtopo</b> to 

In [ ]:
import gempy as gp
import numpy as np
import matplotlib.pyplot as plt
import os
geo_model = gp.create_model('gp_test')

gp.init_data(geo_model, extent=[502536, 600652, 7.458e6, 7.56e6, -3000, 1200],
    resolution = (50,50,50), 
      #path_o = './test_data/output/hams2_orientations.txt',
      path_o = './test_data/output/both_orientations.txt',
      path_i = './test_data/output/hams2_contacts4.txt', default_values=True); #%%

## Show example lithological points

In [ ]:
gp.get_data(geo_model, 'surface_points').head()

## Show example orientations

In [ ]:
gp.get_data(geo_model, 'orientations').head()

## Plot some of this data

In [ ]:
%matplotlib inline
gp.plot.plot_data(geo_model, direction='z');

## Load reprojected topgraphy to model

In [ ]:
fp = './test_data/dtm/hams2_dtm_rp.tif'
geo_model.set_topography(source='gdal',filepath=fp)

## Load stratigraphic Series

f=open('./test_data/tmp/hams2_groups.txt',"r")
contents =f.readlines()
f.close


faults = gp.Faults()
series = gp.Series(faults)
series.df

ngroups=int(contents[0][0])
groups=[]

for i in range (1,int(ngroups)+1):
    groups.append(contents[i].replace("\n",""))
    series.add_series(contents[i].replace("\n",""))

series.delete_series('Default series')

series

## Load surfaces and assign to series

In [ ]:
surfaces = gp.Surfaces(series)


for i in range(0,ngroups):
    f=open('./test_data/tmp/'+groups[i]+'.txt',"r")
    contents =f.readlines()
    nformations=str(contents[0])
    nformations=nformations.split(" ")
    nformations=int(nformations[1].replace("\n",""))

    for j in range (1,int(nformations)+1):
        surfaces.add_surface(contents[j].replace("\n",""))
        d={groups[i]:contents[j].replace("\n","")}
        surfaces.map_series({groups[i]:(contents[j].replace("\n",""))}) #working but no gps
        #gp.map_series_to_surfaces(geo_model, {groups[0]:(contents[j].replace("\n",""))},#XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX<<<<
              #  set_series=True, remove_unused_series=True)
    f.close

surfaces

## Set Interpolation Data

In [ ]:

id_only_one_bool = geo_model.surface_points.df['id'].value_counts() == 1
id_only_one = id_only_one_bool.index[id_only_one_bool]
single_vals = geo_model.surface_points.df[geo_model.surface_points.df['id'].isin(id_only_one)]
for idx, vals in single_vals.iterrows():
    geo_model.add_surface_points(vals['X'], vals['Y'], vals['Z'], vals['surface'])
    
geo_model.update_structure()


In [ ]:
gp.set_interpolation_data(geo_model,
                          compile_theano=True,
                          theano_optimizer='fast_compile',
                          verbose=[])

## Provide summary data on model

In [ ]:
geo_model.additional_data.structure_data

## Calculate Model

In [ ]:


gp.compute_model(geo_model)

## Extract surfaces to visualize in 3D renderers

In [ ]:
gp.plot.plot_section(geo_model, 49, direction='z', show_data=False)

In [ ]:
ver , sim = gp.get_surfaces(geo_model)

## Visualise model

In [ ]:
gp.plot.plot_3D(geo_model)

## Export model to vtk files

from gempy import plot
gp.plot.export_to_vtk(geo_model, path='.\\vtk\\', name=mname+'.vtk', voxels=False, block=None, surfaces=True)